In [2]:
import os
import findspark
findspark.init(os.getenv('SPARK_HOME'))
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [3]:
spark = SparkSession.builder.appName("lrex").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 17:24:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
train = spark.read.format("libsvm").load("sample_linear_regression_data.txt")
train.show()

22/08/16 17:24:44 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [6]:
lrModel = lr.fit(train)

22/08/16 17:24:58 WARN Instrumentation: [0e15666b] regParam is zero, which might cause numerical instability and overfitting.


In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [9]:
train_summary = lrModel.summary

In [10]:
train_summary.r2, train_summary.rootMeanSquaredError, train_summary.meanSquaredError, train_summary.meanAbsoluteError

(0.027839179518600154,
 10.16309157133015,
 103.28843028724194,
 8.145215527783876)

In [11]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")
train, test = all_data.randomSplit([0.85, 0.15])

22/08/16 17:25:05 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [12]:
train.describe().show()
test.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                428|
|   mean|0.17761110417652115|
| stddev| 10.283219363774576|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                 73|
|   mean| 0.7216951689334058|
| stddev| 10.579040983495089|
|    min|-23.487440120936512|
|    max| 19.043184423383824|
+-------+-------------------+



In [13]:
correct_model = lr.fit(train)

22/08/16 17:25:14 WARN Instrumentation: [cbcb22f9] regParam is zero, which might cause numerical instability and overfitting.


In [14]:
test_results = correct_model.evaluate(test)

In [15]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-23.643887741749303|
|-20.274927468672104|
|-15.100357314767463|
|-17.258596645388423|
| -16.04978917944386|
|-18.355839351121073|
| -13.67155229344857|
| -13.77273914603967|
| -11.87295405327146|
| -9.123348718926554|
|-10.288124985582517|
| -12.48801013889489|
|-13.761760379437327|
| -8.604036980937252|
|-13.713691725163905|
|-10.013535522771852|
| -9.149705168784068|
|  -7.11865462437828|
| -8.211349940178035|
|-5.9140396716704675|
+-------------------+
only showing top 20 rows



In [16]:
test_results.r2, test_results.rootMeanSquaredError, test_results.meanSquaredError, test_results.meanAbsoluteError

(-0.01173526390082591,
 10.567799417108603,
 111.67838452024093,
 9.074770874119631)

In [17]:
unlabeled_data = test.select("features")

In [18]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [19]:
predictions = correct_model.transform(unlabeled_data)

In [20]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| 0.15644762081278984|
|(10,[0,1,2,3,4,5,...| 0.39036669439868027|
|(10,[0,1,2,3,4,5,...| -2.3938430421158814|
|(10,[0,1,2,3,4,5,...|   1.535081032339852|
|(10,[0,1,2,3,4,5,...|  0.6739314561315625|
|(10,[0,1,2,3,4,5,...|  2.9962944712883965|
|(10,[0,1,2,3,4,5,...|-0.30457863770413285|
|(10,[0,1,2,3,4,5,...|-0.09434874911909796|
|(10,[0,1,2,3,4,5,...| -1.1048946721206434|
|(10,[0,1,2,3,4,5,...|  -3.344307662106307|
|(10,[0,1,2,3,4,5,...| -1.8422282267054113|
|(10,[0,1,2,3,4,5,...| 0.39365886035963005|
|(10,[0,1,2,3,4,5,...|   1.883592879470485|
|(10,[0,1,2,3,4,5,...| -3.0365126969515748|
|(10,[0,1,2,3,4,5,...|  2.3852757883861235|
|(10,[0,1,2,3,4,5,...| -0.2821304132580677|
|(10,[0,1,2,3,4,5,...| -0.6019174390010132|
|(10,[0,1,2,3,4,5,...|-0.48598185913568015|
|(10,[0,1,2,3,4,5,...|  0.8367289700308063|
|(10,[0,1,2,3,4,5,...| -0.915198